In [6]:
import matplotlib.pyplot as plt
import math
import datetime as dt
import pandas as pd
import numpy as np
from pathlib import Path
import yaml
import requests
import os
import json


In [10]:
with open(Path("config.yaml"), 'r') as f: 
    config = yaml.safe_load(f)

In [11]:
bearer_token = config['token']
# search_url = "https://api.twitter.com/2/tweets/search/all"
search_url = "https://api.twitter.com/2/tweets/search/recent"

In [12]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

def connect_to_endpoint(url, headers, params):
    response = requests.request("GET", search_url, headers=headers, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [14]:
s_ticker = "GME"
count = 10
start_time = None
end_time = None
params = {
    "query": fr"(\${s_ticker}) (lang:en)",
    "max_results": str(count),
    "tweet.fields": "created_at,lang",
}

if start_time:
    # Assign from and to datetime parameters for the API
    params["start_time"] = start_time
if end_time:
    params["end_time"] = end_timea

In [15]:
params

{'query': '(\\$GME) (lang:en)',
 'max_results': '10',
 'tweet.fields': 'created_at,lang'}

In [17]:
headers = create_headers(bearer_token)
json_response = connect_to_endpoint(search_url, headers, params)
print(json.dumps(json_response, indent=4, sort_keys=True))

200
{
    "data": [
        {
            "created_at": "2021-06-29T21:07:14.000Z",
            "id": "1409981811963445255",
            "lang": "en",
            "text": "RT @unusual_whales: \ud83d\udea8 BREAKING NEWS \ud83d\udea8\n\n@Trading212 states to users there needs to be \"consent\" to users to lend out $GME shares.\n\nElsewise,\u2026"
        },
        {
            "created_at": "2021-06-29T21:07:09.000Z",
            "id": "1409981791495241729",
            "lang": "en",
            "text": "RT @CannabisCricket: $ALF $ALFIW 22.50 HOD $25 COMING NEXT NEXT LEG UP\n\nHIGH SHORT % SHORTS TRAPPED CANT WIN\n\nBIG SQUEEZE IS YET TO COME\n\nA\u2026"
        },
        {
            "created_at": "2021-06-29T21:07:07.000Z",
            "id": "1409981781789523984",
            "lang": "en",
            "text": "RT @CannabisCricket: $ALF $ALFIW 22.50 HOD $25 COMING NEXT NEXT LEG UP\n\nHIGH SHORT % SHORTS TRAPPED CANT WIN\n\nBIG SQUEEZE IS YET TO COME\n\nA\u2026"
        },
        {


In [26]:
from datetime import datetime 
def get_data(tweet):
    """Gets twitter data from API request"""
    if "+" in tweet["created_at"]:
        s_datetime = tweet["created_at"].split(" +")[0]
    else:
        s_datetime = datetime.strptime(tweet["created_at"].split(".")[0], "%Y-%m-%dT%H:%M:%S").strftime(
            "%Y-%m-%d %H:%M:%S"
        )

    if "full_text" in tweet.keys():
        s_text = tweet["full_text"]
    else:
        s_text = tweet["text"]

    data = {"created_at": s_datetime, "text": s_text}
    return data
    
def clean_tweet(tweet: str, s_ticker: str) -> str:
    """Cleans tweets to be fed to sentiment model"""
    whitespace = re.compile(r"\s+")
    web_address = re.compile(r"(?i)http(s):\/\/[a-z0-9.~_\-\/]+")
    ticker = re.compile(fr"(?i)@{s_ticker}(?=\b)")
    user = re.compile(r"(?i)@[a-z0-9_]+")

    tweet = whitespace.sub(" ", tweet)
    tweet = web_address.sub("", tweet)
    tweet = ticker.sub(s_ticker, tweet)
    tweet = user.sub("", tweet)

    return tweet

In [27]:
df_tweets = pd.DataFrame()
for tweet in json_response["data"]:
    row = get_data(tweet)
    df_tweets = df_tweets.append(row, ignore_index=True)

In [28]:
df_tweets

,created_at,text
0,2021-06-29 21:07:14,RT @unusual_whales: 🚨 BREAKING NEWS 🚨\n\n@Trad...
1,2021-06-29 21:07:09,RT @CannabisCricket: $ALF $ALFIW 22.50 HOD $25...
2,2021-06-29 21:07:07,RT @CannabisCricket: $ALF $ALFIW 22.50 HOD $25...
3,2021-06-29 21:07:06,RT @masked_investor: So since 002 has been liv...
4,2021-06-29 21:06:58,RT @GMEshortsqueeze: $GME I wonder if these ar...
5,2021-06-29 21:06:55,We got someone’s attention #AMC #GME https://t...
6,2021-06-29 21:06:50,RT @WSB_Degenerate: $CLOV\n\nWhen borrow fees ...
7,2021-06-29 21:06:44,RT @cvpayne: .@Trading212 there is some confus...
8,2021-06-29 21:06:39,$XELA short squeeze for the rest of the week!!...
9,2021-06-29 21:06:39,RT @CannabisCricket: $BSQR $MRIN $AUUD $ALF $X...
